In [1]:
import pandas as pd
from ortools.linear_solver import pywraplp as OR

In [2]:
items = pd.read_csv('knapsack_example.csv', index_col=0)

In [3]:
def knapsack(items, capacity, integer = False):
    """Model for solving the Knapsack problem.
    
    Args:
        items (pd.DataFrame): A table indexd by items with a column for value and weight
        capcity (int): An integer-capacity for the knapsack
        integer (bool): True if the variables should be integer. False otherwise.
    """
    ITEMS = list(items.index)        # set of items
    v = items.to_dict()['value']     # value for each item 
    w = items.to_dict()['weight']    # weight for each item
    W = capacity                     # capacity of the knapsack
    
    # define model
    m = OR.Solver('knapsack', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    # decision variables
    x = {}    
    for i in ITEMS:
        if integer:
            x[i] = m.IntVar(0, 1, 'x_%d' % (i)) 
        else:
            x[i] = m.NumVar(0, 1, 'x_%d' % (i)) 
        
    # define objective function here
    m.Maximize(sum(v[i]*x[i] for i in ITEMS))
    
    # weight must not exceed capacity
    m.Add(sum(w[i]*x[i] for i in ITEMS) <= W)
    
    return m, x

In [4]:
def solve(m):
    m.Solve()
    print('Solution:')
    print('Objective value =', m.Objective().Value())
    for var in m.variables():
        print(var.name(), ':',  var.solution_value())

In [5]:
m, x = knapsack(items, 18)
solve(m)

Solution:
Objective value = 70.0
x_1 : 1.0
x_2 : 0.6666666666666666
x_3 : 0.0
x_4 : 0.0
x_5 : 0.0
x_6 : 0.0
x_7 : 0.0


In [6]:
m, x = knapsack(items, 18, integer=True)
solve(m)

Solution:
Objective value = 64.0
x_1 : 1.0
x_2 : 0.0
x_3 : 0.0
x_4 : 1.0
x_5 : 0.0
x_6 : 0.0
x_7 : 0.0
